In [23]:
from PIL import Image
import numpy as np
import os
import open3d as o3d


In [24]:
def changeDep480to64(np_depth):
    lines = 16
    rows , cols = np_depth.shape
    pixel_y = [round(i * int(rows/lines)) for i in range(lines)]
    mask = np.ones((rows ,cols), dtype='bool')
    mask[pixel_y,:] = False
    np_depth[mask] = 0.0
    return np_depth

def simulateArc2(depth_data):
    """同一个原点按半径不同划分
    """
    lines = 16
    rows , cols = depth_data.shape
    pixel_y = np.array([round(i * int(rows/lines)) for i in range(lines)])
    origin = (rows, cols/2)
    distance = origin[0] - pixel_y
    mask = np.zeros((rows ,cols), dtype=np.bool)
    for i in range(rows):
        for j in range(cols):
            if round(np.sqrt((i-origin[0])**2 + (j-origin[1])**2)) in distance:
                mask[i][j] = True
                
    return mask


def simulateArc1(depth_data):
    lines = 16
    rows , cols = depth_data.shape
    coodi_y = np.array([round(i * int(rows/lines) + cols/2) for i in range(lines)])
    mask = np.zeros((rows ,cols), dtype=np.bool)
    for i in range(rows):
        for j in range(cols):
            for coodi_ in coodi_y:
                if round(np.sqrt((i-coodi_)**2 + (j-cols/2)**2)) == round(cols/2 * np.sqrt(2)):
                    mask[i][j] = True
    
    
    return mask

def simulateArc3(depth_data):
    lines = 16
    rows , cols = depth_data.shape
    height = cols - (np.sqrt(2)- 1 ) * cols / 2
    coodi_y = np.array([round(i * int(height / lines)) for i in range(lines)])
    mask = np.zeros((rows ,cols), dtype=np.bool)
    
    for i in range(rows):
        for j in range(cols):
            for coodi_ in coodi_y:
                if round(np.sqrt((i-coodi_)**2 + (j-cols/2)**2)) == round(cols/2 * np.sqrt(2)):
                    mask[i][j] = True
    
    return mask

def simulateArc4(depth_data):
    lines = 16
    rows , cols = depth_data.shape
    radius = cols / np.sin(np.radians(45))
    height = cols + cols / np.tan(np.radians(45))
    origin = (height, cols/2)
    
    radius2 = cols / np.sin(np.radians(45)) + 100
    mask = np.zeros((rows ,cols), dtype=np.bool)
    
    for i in range(rows):
        for j in range(cols):
            if round(np.sqrt((i-origin[0])**2 + (j-origin[1])**2)) == round(radius):
                mask[i][j] = True
            elif round(np.sqrt((i-origin[0])**2 + (j-origin[1])**2)) == round(radius2):
                mask[i][j] = True
            
    return mask




def Function(depth_data):
    # depth_data = changeDep480to64(depth_data)
    mask = simulateArc4(depth_data)
    
    depth_data = depth_data.astype(float)
    depth_data[~mask] = np.nan
    
    
    return depth_data

#   =============================================================

def cloud_16(point_cloud):
    cloud_16 = np.array([0.,0.,0.])
    lines = []
    point_cloud[:,2] += 1.7
    z_min = point_cloud[:,2].min()
    z_max = point_cloud[:,2].max()
    # 16等分
    boundaries = np.linspace(z_min, z_max, num=17)
    for boundarie in boundaries[:-1]:
        condition = (point_cloud[:,2] > boundarie) & (point_cloud[:,2] < boundarie+0.01)
        cloud_16 = np.vstack((cloud_16,point_cloud[condition]))
        lines.append(point_cloud[condition])
    print(f'len(lines):{len(lines)}')
        
        
    cloud_16 = cloud_16[1:]
    # cloud_16[:,2] -= 1.7
    print(f"cloud_16.shape:{cloud_16.shape}")
    return point_cloud

def voxel_down_sample(point_cloud):
    sparse_cloud = o3d.geometry.PointCloud()
    sparse_cloud.points = o3d.utility.Vector3dVector(point_cloud)
    voxel_size = 0.05
    voxel_cloud = sparse_cloud.voxel_down_sample(voxel_size)
    output_cloud = np.asarray(voxel_cloud.points)
    return output_cloud

def PointFunction(point_cloud):
    # cloud_16(point_cloud)
    # point_cloud = voxel_down_sample(point_cloud)
    return point_cloud

In [25]:
DEPTH_SCALE_M = 0.001

# 以下示例假设相机参数为 fx, fy（焦距），cx, cy（光学中心）
fx = 392.323 
fy = 392.323
cx = 319.158 
cy = 240.853

## D435
# fx = 392.323 
# fy = 392.323
# cx = 322.776 
# cy = 239.792

depth_image = Image.open('image_93.png')
depth_data = np.array(depth_image)
depth_data = Function(depth_data)
# 创建一个空的点云列表，用于存储点的坐标
point_cloud = []

# 遍历深度图像的每个像素
for y in range(depth_data.shape[0]):
    for x in range(depth_data.shape[1]):
        # 获取深度值
        depth = depth_data[y, x] * DEPTH_SCALE_M

        # 计算对应的点的坐标（假设 Z 轴为深度，X 和 Y 轴为图像平面）
        if depth > 0:
            point_x = (x - cx) * depth / fx
            point_y = (y - cy) * depth / fy
            point_z = depth

            # 将点的坐标添加到点云列表中
            point_cloud.append([point_x, point_y, point_z])

# 绕 Y 轴顺时针旋转 90 度
angle_y = np.radians(90)  # 将角度转换为弧度
rotation_y = np.array([[np.cos(angle_y), 0, np.sin(angle_y)],
                    [0, 1, 0],
                    [-np.sin(angle_y), 0, np.cos(angle_y)]])
rotated_points = np.dot(point_cloud, rotation_y.T)
# 绕 X 轴逆时针旋转 90 度
angle_x = np.radians(-90)  # 将角度转换为弧度
rotation_x = np.array([[1, 0, 0],
                    [0, np.cos(angle_x), -np.sin(angle_x)],
                    [0, np.sin(angle_x), np.cos(angle_x)]])
rotated_points = np.dot(rotated_points, rotation_x.T)

nan_mask = np.isnan(rotated_points).any(axis=1)
point_cloud = rotated_points[~nan_mask]
# 步骤 4：可选地，保存点云数据到文件或进行可视化
# 这里不提供具体的点云保存或可视化代码，因为涉及到不同的库和工具。


c:\Users\junwe\anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [26]:

# 将点云列表转换为 NumPy 数组
point_cloud = np.array(point_cloud)

point_cloud = PointFunction(point_cloud)

root, extension = os.path.splitext('image_93.png')
np.save('point_'+root[6:]+'.npy', point_cloud)

print(point_cloud.shape)
    

(1312, 3)


In [27]:

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
pcd.paint_uniform_color([1.0, 0.5, 0.0])  # 设置橙色

# pcd2 = o3d.geometry.PointCloud()
# pcd2.points = o3d.utility.Vector3dVector(unproce_point_cloud)
# pcd2.paint_uniform_color([1.0, 0.0, 0.0])  # 设置红色

lidar_point_cloud  = np.load('lidarpoint_93.npy')
lidar_point_cloud[:,2] += 0.22
pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(lidar_point_cloud)
pcd3.paint_uniform_color([0.0, 0.5, 0.3])  # 设置红色


coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

# 创建一个场景并将点云、包围框和坐标系添加到场景中
scene = o3d.visualization.Visualizer()
scene.create_window()
scene.add_geometry(pcd)        # 添加点云
# scene.add_geometry(pcd2)        # 添加点云
scene.add_geometry(pcd3)        # 添加点云
scene.add_geometry(coord_frame)  # 添加坐标系


# 设置视角
view_control = scene.get_view_control()
view_control.set_lookat([0, 0, 0])  # 设置视点
view_control.set_up([1, 0, 0])     # 设置视角上方的方向

# 显示场景
scene.run()
scene.destroy_window()